In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

In [2]:
x_tr_resample = pd.read_csv('../../Data/clean/X_train_smote.csv')
X_test = pd.read_csv('../../Data/clean/X_test.csv')
y_tr_resample = np.loadtxt("../../Data/clean/y_train_smote.csv", delimiter=",")
y_test = np.loadtxt("../../Data/clean/y_test.csv", delimiter=",")

In [3]:
skf_grid = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [4]:
base_params = {
    'task_type': 'GPU',
    'devices': '0:1',  # Gunakan GPU pertama
    'gpu_ram_part': 0.95,  # Gunakan 95% memory GPU yang tersedia
}

In [5]:
catboost_params = {
    'iterations': [500],             # Kurangi iterasi untuk mempercepat proses
    'depth': [6],                    # Gunakan kedalaman pohon tunggal
    'learning_rate': [0.05],         # Gunakan 1 nilai yang moderat
    'l2_leaf_reg': [3],              # Tetapkan 1 nilai regularisasi
    'border_count': [128],           # Nilai optimal untuk GPU
    'bootstrap_type': ['Bernoulli'], # Pertahankan bootstrap Bernoulli
    'subsample': [0.8],              # Satu nilai subsample
    'grow_policy': ['SymmetricTree'],# Pertahankan kebijakan cepat
    'eval_metric': ['AUC'],          # Gunakan AUC sebagai metrik evaluasi
    'od_type': ['Iter'],             # Early stopping
    'od_wait': [30],                 # Kurangi iterasi tanpa peningkatan
}


In [6]:
init_params = base_params.copy()

In [7]:
model = CatBoostClassifier(**init_params)

In [8]:
grid_search = GridSearchCV(
        estimator=model,
        param_grid=catboost_params,
        cv=skf_grid,
        n_jobs=1,  # Gunakan 1 karena kita menggunakan GPU
        verbose=1,
        scoring='f1_weighted',
)

In [9]:
train_pool = Pool(x_tr_resample, y_tr_resample)

In [10]:
grid_search.fit(x_tr_resample, y_tr_resample)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 381ms	remaining: 3m 10s
1:	total: 619ms	remaining: 2m 34s
2:	total: 726ms	remaining: 2m
3:	total: 817ms	remaining: 1m 41s
4:	total: 917ms	remaining: 1m 30s
5:	total: 1.03s	remaining: 1m 24s
6:	total: 1.12s	remaining: 1m 18s
7:	total: 1.21s	remaining: 1m 14s
8:	total: 1.3s	remaining: 1m 11s
9:	total: 1.39s	remaining: 1m 8s
10:	total: 1.52s	remaining: 1m 7s
11:	total: 1.66s	remaining: 1m 7s
12:	total: 1.76s	remaining: 1m 5s
13:	total: 1.92s	remaining: 1m 6s
14:	total: 2.05s	remaining: 1m 6s
15:	total: 2.14s	remaining: 1m 4s
16:	total: 2.23s	remaining: 1m 3s
17:	total: 2.32s	remaining: 1m 2s
18:	total: 2.4s	remaining: 1m
19:	total: 2.48s	remaining: 59.5s
20:	total: 2.55s	remaining: 58.3s
21:	total: 2.64s	remaining: 57.3s
22:	total: 2.72s	remaining: 56.4s
23:	total: 2.79s	remaining: 55.4s
24:	total: 2.9s	remaining: 55s
25:	total: 2.98s	remaining: 54.4s
26:	total: 3.08s	remaining: 53.9s
27:	total: 3.18s	remaining: 53.7s
28:	total: 3.28s	remaining: 53.2s
29:	total: 3.36s	remaining:

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 85ms	remaining: 42.4s
1:	total: 178ms	remaining: 44.3s
2:	total: 276ms	remaining: 45.7s
3:	total: 368ms	remaining: 45.7s
4:	total: 459ms	remaining: 45.4s
5:	total: 554ms	remaining: 45.6s
6:	total: 650ms	remaining: 45.8s
7:	total: 720ms	remaining: 44.3s
8:	total: 816ms	remaining: 44.5s
9:	total: 911ms	remaining: 44.6s
10:	total: 1.01s	remaining: 45s
11:	total: 1.08s	remaining: 44s
12:	total: 1.17s	remaining: 43.9s
13:	total: 1.3s	remaining: 45.3s
14:	total: 1.47s	remaining: 47.5s
15:	total: 1.53s	remaining: 46.5s
16:	total: 1.6s	remaining: 45.6s
17:	total: 1.69s	remaining: 45.3s
18:	total: 1.79s	remaining: 45.2s
19:	total: 1.89s	remaining: 45.3s
20:	total: 1.98s	remaining: 45.2s
21:	total: 2.07s	remaining: 44.9s
22:	total: 2.17s	remaining: 44.9s
23:	total: 2.26s	remaining: 44.9s
24:	total: 2.36s	remaining: 44.8s
25:	total: 2.45s	remaining: 44.7s
26:	total: 2.55s	remaining: 44.6s
27:	total: 2.65s	remaining: 44.7s
28:	total: 2.74s	remaining: 44.5s
29:	total: 2.82s	remaining: 44.

...

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 99.4ms	remaining: 49.6s
1:	total: 201ms	remaining: 50.2s
2:	total: 299ms	remaining: 49.5s
3:	total: 420ms	remaining: 52.1s
4:	total: 503ms	remaining: 49.8s
5:	total: 579ms	remaining: 47.6s
6:	total: 674ms	remaining: 47.4s
7:	total: 791ms	remaining: 48.6s
8:	total: 890ms	remaining: 48.6s
9:	total: 989ms	remaining: 48.5s
10:	total: 1.13s	remaining: 50.4s
11:	total: 1.3s	remaining: 53s
12:	total: 1.38s	remaining: 51.9s
13:	total: 1.45s	remaining: 50.2s
14:	total: 1.52s	remaining: 49.1s
15:	total: 1.61s	remaining: 48.6s
16:	total: 1.71s	remaining: 48.6s
17:	total: 1.81s	remaining: 48.5s
18:	total: 1.9s	remaining: 48.2s
19:	total: 2.01s	remaining: 48.2s
20:	total: 2.1s	remaining: 47.9s
21:	total: 2.19s	remaining: 47.7s
22:	total: 2.3s	remaining: 47.7s
23:	total: 2.41s	remaining: 47.8s
24:	total: 2.5s	remaining: 47.6s
25:	total: 2.6s	remaining: 47.4s
26:	total: 2.68s	remaining: 46.9s
27:	total: 2.77s	remaining: 46.7s
28:	total: 2.86s	remaining: 46.5s
29:	total: 2.98s	remaining: 46.

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 104ms	remaining: 52s
1:	total: 220ms	remaining: 54.8s
2:	total: 316ms	remaining: 52.4s
3:	total: 431ms	remaining: 53.5s
4:	total: 524ms	remaining: 51.9s
5:	total: 599ms	remaining: 49.3s
6:	total: 696ms	remaining: 49s
7:	total: 786ms	remaining: 48.3s
8:	total: 882ms	remaining: 48.1s
9:	total: 988ms	remaining: 48.4s
10:	total: 1.07s	remaining: 47.8s
11:	total: 1.17s	remaining: 47.7s
12:	total: 1.27s	remaining: 47.7s
13:	total: 1.37s	remaining: 47.5s
14:	total: 1.47s	remaining: 47.6s
15:	total: 1.57s	remaining: 47.6s
16:	total: 1.68s	remaining: 47.8s
17:	total: 1.78s	remaining: 47.8s
18:	total: 1.89s	remaining: 47.9s
19:	total: 2.01s	remaining: 48.3s
20:	total: 2.12s	remaining: 48.4s
21:	total: 2.24s	remaining: 48.7s
22:	total: 2.34s	remaining: 48.6s
23:	total: 2.44s	remaining: 48.5s
24:	total: 2.55s	remaining: 48.4s
25:	total: 2.65s	remaining: 48.4s
26:	total: 2.75s	remaining: 48.1s
27:	total: 2.85s	remaining: 48.1s
28:	total: 2.96s	remaining: 48.1s
29:	total: 3.07s	remaining: 

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=<catboost.core.CatBoostClassifier object at 0x0000028C5F3C2330>,
             n_jobs=1,
             param_grid={'bootstrap_type': ['Bernoulli'], 'border_count': [128],
                         'depth': [6], 'eval_metric': ['AUC'],
                         'grow_policy': ['SymmetricTree'], 'iterations': [500],
                         'l2_leaf_reg': [3], 'learning_rate': [0.05],
                         'od_type': ['Iter'], 'od_wait': [30],
                         'subsample': [0.8]},
             scoring='f1_weighted', verbose=1)

In [11]:
# Get best parameters
best_params = grid_search.best_params_

In [12]:


# Create new model with best parameters
best_model = CatBoostClassifier(**best_params, silent=True, random_state=42)



In [13]:
skf_eval = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

accuracies = []
recalls = []
precisions = []
f1_scores = []
roc_aucs = []

for fold, (train_idx, val_idx) in enumerate(skf_eval.split(x_tr_resample, y_tr_resample)):
    # Split data
    X_train_fold = x_tr_resample.iloc[train_idx]
    y_train_fold = y_tr_resample[train_idx]
    X_val_fold = x_tr_resample.iloc[val_idx]
    y_val_fold = y_tr_resample[val_idx]
    
    # Train model
    best_model.fit(X_train_fold, y_train_fold)
    
    # Make predictions
    y_pred_fold = best_model.predict(X_val_fold)
    y_pred_proba_fold = best_model.predict_proba(X_val_fold)[:, 1]
    
    # Calculate metrics
    accuracies.append(accuracy_score(y_val_fold, y_pred_fold))
    recalls.append(recall_score(y_val_fold, y_pred_fold, average='weighted'))
    precisions.append(precision_score(y_val_fold, y_pred_fold, average='weighted'))
    f1_scores.append(f1_score(y_val_fold, y_pred_fold, average='weighted'))
    roc_aucs.append(roc_auc_score(y_val_fold, y_pred_proba_fold))

In [14]:
best_model.fit(x_tr_resample, y_tr_resample)
y_pred_test = best_model.predict(X_test)

y_pred_test_proba = best_model.predict_proba(X_test)[:, 1]

y_train_pred = best_model.predict(x_tr_resample)

In [15]:
print("CatBoost Results:")
print("-" * 50)
print("Best Parameters:", best_params)
print("\nCross-validation Results (10-fold):")
print(f"Accuracy    : {np.mean(accuracies)*100:.2f}% (+/- {np.std(accuracies)*100:.2f}%)")
print(f"Recall      : {np.mean(recalls)*100:.2f}% (+/- {np.std(recalls)*100:.2f}%)")
print(f"Precision   : {np.mean(precisions)*100:.2f}% (+/- {np.std(precisions)*100:.2f}%)")
print(f"F1-Score    : {np.mean(f1_scores)*100:.2f}% (+/- {np.std(f1_scores)*100:.2f}%)")

print("\nTrain Set Results:")
print(f"Accuracy    : {accuracy_score(y_tr_resample, y_train_pred)*100:.2f}%")
print(f"Recall      : {recall_score(y_tr_resample, y_train_pred, average='weighted')*100:.2f}%")
print(f"Precision   : {precision_score(y_tr_resample, y_train_pred, average='weighted')*100:.2f}%")
print(f"F1-Score    : {f1_score(y_tr_resample, y_train_pred, average='weighted')*100:.2f}%")

print("\nTest Set Results:")
print(f"Accuracy    : {accuracy_score(y_test, y_pred_test)*100:.2f}%")
print(f"Recall      : {recall_score(y_test, y_pred_test, average='weighted')*100:.2f}%")
print(f"Precision   : {precision_score(y_test, y_pred_test, average='weighted')*100:.2f}%")
print(f"F1-Score    : {f1_score(y_test, y_pred_test, average='weighted')*100:.2f}%")
print(f"ROC AUC     : {roc_auc_score(y_test, y_pred_test_proba)*100:.2f}%")
print("-" * 50)

CatBoost Results:
--------------------------------------------------
Best Parameters: {'bootstrap_type': 'Bernoulli', 'border_count': 128, 'depth': 6, 'eval_metric': 'AUC', 'grow_policy': 'SymmetricTree', 'iterations': 500, 'l2_leaf_reg': 3, 'learning_rate': 0.05, 'od_type': 'Iter', 'od_wait': 30, 'subsample': 0.8}

Cross-validation Results (10-fold):
Accuracy    : 99.43% (+/- 0.14%)
Recall      : 99.43% (+/- 0.14%)
Precision   : 99.43% (+/- 0.14%)
F1-Score    : 99.43% (+/- 0.14%)

Train Set Results:
Accuracy    : 99.93%
Recall      : 99.93%
Precision   : 99.93%
F1-Score    : 99.93%

Test Set Results:
Accuracy    : 99.34%
Recall      : 99.34%
Precision   : 99.34%
F1-Score    : 99.34%
ROC AUC     : 99.93%
--------------------------------------------------
